# 간단한 파이프라인 예제 실습

## 1. 필요한 라이브러리를 import

In [26]:
import kfp
from kfp.components import create_component_from_func
from kfp.dsl import pipeline
from kfp.compiler import Compiler
import os
from kakaocloud_kbm import KbmPipelineClient

## 2. Kubeflow Pipelines에 접속하기 위한 환경 변수를 설정

In [27]:
os.environ["KUBEFLOW_HOST"] = "https://대시보드 접속한 로드밸런서의 Public IP/"
os.environ["KUBEFLOW_USERNAME"] = "Kubeflow 대시보드 접속 아이디(이메일)"
os.environ["KUBEFLOW_PASSWORD"] = "Kubeflow 대시보드 접속 비밀번호"

## 3-1. 컴포넌트 정의 (덧셈 함수를 정의 후 Kubeflow 컴포넌트로 변환)

In [28]:
# 간단한 컴포넌트 정의
def add(a: float, b: float) -> float:
    return a + b

# Kubeflow 컴포넌트로 변환
add_op = create_component_from_func(add, base_image='python:3.8', packages_to_install=[])

## 3-2. 컴포넌트 정의 (곱셈 함수를 정의 후 Kubeflow 컴포넌트로 변환)

In [29]:
def multiply(a: float, b: float) -> float:
    return a * b

multiply_op = create_component_from_func(multiply, base_image='python:3.8', packages_to_install=[])

## 4. 두 개의 컴포넌트를 이용하여 **파이프라인을 정의**

In [30]:
@pipeline(name='Addition and Multiplication Pipeline', description='A simple pipeline that adds and multiplies numbers.')
def math_pipeline(a: float , b: float , c: float ):
    add_task = add_op(a, b)
    multiply_task = multiply_op(add_task.output, c)

## 5. 파이프라인을 정의한 후 이를 **컴파일**하여 YAML 파일로 저장

In [ ]:
# 파이프라인 컴파일
pipeline_file = 'math_pipeline.yaml'
Compiler().compile(math_pipeline, pipeline_file)
print(f"Pipeline definition is saved to {pipeline_file}")

## 6. SDK를 통해 파이프라인을 실행
실험 이름과 실행 이름을 정의하고, 파이프라인의 매개변수를 설정

In [ ]:
experiment_name = math_pipeline.__name__ + ' test experiment'
run_name = math_pipeline.__name__ + ' run'
arguments = {
    "a": 1,
    "b": 2,
    "c": 3
}

client = KbmPipelineClient()
client.create_run_from_pipeline_func(
    math_pipeline, 
    experiment_name=experiment_name, 
    run_name=run_name, 
    arguments=arguments
)